In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
raw_dataset = pd.read_csv('/content/drive/MyDrive/Rice Google Drive/Research/Misc/nasim_ML.csv', sep = ',')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Rice Google Drive/Research/Misc/nasim_ML.csv'

In [ ]:
raw_dataset.drop(['time (ns)','Explosive'],axis=1,inplace=True)

In [ ]:
raw_dataset.columns = ['\u03B2', '\u03B4','$\mathregular{N_e}$','$\mathregular{T_w}$*','$\mathregular{T_{Ar}}$*']

In [ ]:
raw_dataset.columns

**Data Wrangling**

In [ ]:
raw_dataset.isnull().sum()

In [ ]:
raw_dataset.dropna(inplace=True)

In [ ]:
raw_dataset.isnull().sum()

**Correlation**

In [ ]:
image = plt.figure(figsize=(16,10), dpi = 600)
sns.heatmap(raw_dataset.corr(), annot = True)
plt.show()
image.savefig('Correlation.png',  bbox_inches="tight")

**Split Data into X & Y**

In [ ]:
raw_dataset.columns = ['HR', 'thickness','wetting','wall','liquid']

In [ ]:
notoutput = "liquid"
output = "wall"

In [ ]:
X = raw_dataset.drop([output, notoutput], axis = 1)
Y = raw_dataset.iloc[:,3:5]

In [ ]:
X

In [ ]:
Y

**Pre-Processing of Dataset**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

In [ ]:
# Applying Transformation
scaler_X = scaler.fit(X)
scaled_X = scaler.transform(X)

In [ ]:
# Applying Transformation
scaler_Y = scaler.fit(Y)
scaled_Y = scaler.transform(Y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(scaled_X, scaled_Y, test_size = 0.2, random_state = 1)

**ANN Regression**

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from keras import regularizers

In [ ]:
def build_model():
  model = Sequential()
  # Input Layer
  # units = num. of neurons, input shape = shape of features
  lambdaa = 1e-5
  model.add(Dense(units = 16, activation='relu',input_shape = [len(X.keys())],activity_regularizer=regularizers.l2(lambdaa)))

  # Hidden Layer - I
  model.add(Dense(units = 32, activation='relu',activity_regularizer=regularizers.l2(lambdaa)))

  # Hidden Layer - II
  model.add(Dense(units = 32, activation='relu',activity_regularizer=regularizers.l2(lambdaa)))

  # Output Layer
  # Activation Function - Linear
  model.add(Dense(units = 2))

  # Optimizers
  # Learning rate = 'alpha'
  optimizers = Adam(learning_rate= 0.001)

  model.compile(loss = 'mean_squared_error', optimizer = optimizers)
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
# Epochs = Number of Iterations, Batch Size = number of samples per execution
# validation_split = 15% of overall training samples to validate algorithm parallely to avoid overfitting of algorithm
#callback = EarlyStopping(monitor='val_loss', patience=1000)
history = model.fit(X_train, Y_train, epochs = 600, validation_split = 0.2)

In [ ]:
pd.DataFrame(history.history)[['loss', 'val_loss']].plot(figsize = (8,6))
# plt.ylim(top=100)
plt.ylim(bottom=0)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(Y_test, predictions)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(Y_test,predictions)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(Y_test,predictions)

In [ ]:
# model.weights
predicted_data = model.predict(scaled_X)
r2_score(scaled_Y, predicted_data)

In [ ]:
mean_absolute_error(scaled_Y,predicted_data)

In [ ]:
model.save('/content/drive/MyDrive/Rice Google Drive/Research/Misc/wall_ML_neural/model.h5')

In [ ]:
image2 = plt.figure(figsize=(10,10), dpi = 600)
plt.scatter(Y.iloc[:,0], predicted_Y[:,0])
gg = r2_score(Y.iloc[:,0], predicted_Y[:,0])
plt.text(175, 325, '$\mathregular{R^2 =}$'+str(round(gg,4)))
plt.plot(Y.iloc[:,0],Y.iloc[:,0])
plt.xlabel('$\mathregular{T_w}$* from MD')
plt.ylabel('$\mathregular{T_w}$* from ANN')
plt.show()
image2.savefig('T_w.png',  bbox_inches="tight")

In [ ]:
image3 = plt.figure(figsize=(10,10), dpi = 600)
plt.scatter(Y.iloc[:,1], predicted_Y[:,1])
plt.plot(Y.iloc[:,1],Y.iloc[:,1])
gg = r2_score(Y.iloc[:,1], predicted_Y[:,1])
plt.text(125, 145, '$\mathregular{R^2 =}$'+str(round(gg,4)))
plt.xlabel('$\mathregular{T_{Ar}}$* from MD')
plt.ylabel('$\mathregular{T_{Ar}}$* from ANN')
plt.show()
image3.savefig('T_l.png',  bbox_inches="tight")

In [ ]:
classification_results=pd.read_csv('/content/drive/MyDrive/Rice Google Drive/Research/Misc/classification_results.csv', sep = ',')

In [ ]:
classification_results = classification_results.iloc[:,1:5]

In [ ]:
classification_results.iloc[:,2]=0.5

In [ ]:
classification_results.iloc[:,0:3]

In [ ]:
scaler_X_grid = scaler.fit(classification_results.iloc[:,0:3])
scaled_X_grid = scaler_X_grid.transform(classification_results.iloc[:,0:3])

In [ ]:
scaled_Z = model.predict(scaled_X_grid)

In [ ]:
scaler_Y = scaler.fit(Y)

In [ ]:
Z = scaler_Y.transform(scaled_Z)

In [ ]:
X_grid, Y_grid = np.meshgrid(np.linspace(40, 320, 100), np.linspace(3, 6, 100))

In [ ]:
Z_wall = Z[:,0].reshape(X_grid.shape)

In [ ]:
image4, axs = plt.subplots(1,1,figsize=(15,15))
# fig,ax=plt.subplots(1,1)
cp = ax.contourf(X_grid, Y_grid, Z_wall)
image4.colorbar(cp)
ax.set_xlabel(r'$\beta \times 10^9 Ks^{-1}$')
ax.set_ylabel(r'$\delta$ (nm)')
plt.savefig('jp.jpg')
plt.show()